In [1]:
import os
from sklearn import preprocessing
import sys
import numpy as np
import pandas as pd
import pickle
from azureml.dataprep import package
sys.path.append(".")
sys.path.append("..")

removedWordsList = (['xxxxx1'])


def removeNonEnglish(text, englishWords):
    global removedWordsList
    wordList = text.split()
    if len(wordList) == 0:
        return " "
    y = np.array(wordList)
    x = np.array(englishWords)
    index = np.arange(len(englishWords))
    sorted_index = np.searchsorted(x, y)
    yindex = np.take(index, sorted_index, mode="clip")
    mask = x[yindex] != y
    maskedArr = np.ma.array(yindex, mask=mask).compressed()
    result = x[maskedArr]
    text = np.array2string(result)\
        .replace("\'", "")\
        .replace("[", "")\
        .replace("]", "")\
        .replace("\n", "")\
        .replace("\r", "")

    # Logging removed words
    removedWords = set(wordList)-set(result)
    removedWordsList += set(list(removedWords))-set(removedWordsList)
    return text


def encryptSingleColumn(data):
    le = preprocessing.LabelEncoder()
    le.fit(data)
    return le.transform(data)


def encryptColumnsCollection(data, columnsToEncrypt):
    for column in columnsToEncrypt:
        data[column] = encryptSingleColumn(data[column])
    return data


def removeString(data, regex):
    return data.str.lower().str.replace(regex.lower(), ' ')


def cleanDataset(dataset, columnsToClean, regexList):
    for column in columnsToClean:
        for regex in regexList:
            dataset[column] = removeString(dataset[column], regex)
    return dataset


def getRegexList():
    regexList = []
    regexList += ['From:(.*)\r\n']  # from line
    # regexList += ['RITM[0-9]*'] # request id
    # regexList += ['INC[0-9]*'] # incident id
    # regexList += ['TKT[0-9]*'] # ticket id
    regexList += ['Sent:(.*)\r\n']  # sent to line
    regexList += ['Received:(.*)\r\n']  # received data line
    regexList += ['To:(.*)\r\n']  # to line
    regexList += ['CC:(.*)\r\n']  # cc line
    regexList += ['The information(.*)infection']  # footer
    regexList += ['Endava Limited is a company(.*)or omissions']  # footer
    regexList += ['The information in this email is confidential and may be legally(.*)interference if you are not the intended recipient']  # footer
    regexList += ['\[cid:(.*)]']  # images cid
    regexList += ['https?:[^\]\n\r]+']  # https & http
    regexList += ['Subject:']
    # regexList += ['[\w\d\-\_\.]+@[\w\d\-\_\.]+']  # emails
    # regexList += ['[0-9][\-0–90-9 ]+']  # phones
    # regexList += ['[0-9]']  # numbers
    # regexList += ['[^a-zA-z 0-9]+']  # anything that is not a letter
    # regexList += ['[\r\n]']  # \r\n
    # regexList += [' [a-zA-Z] ']  # single letters
    # regexList += [' [a-zA-Z][a-zA-Z] ']  # two-letter words
    # regexList += ["  "]  # double spaces

    regexList += ['^[_a-z0-9-]+(\.[_a-z0-9-]+)*@[a-z0-9-]+(\.[a-z0-9-]+)*(\.[a-z]{2,4})$']
    regexList += ['[\w\d\-\_\.]+ @ [\w\d\-\_\.]+']
    regexList += ['Subject:']
    regexList += ['[^a-zA-Z]']

    return regexList


In [2]:
####################
# Use this with AML Workbench to load data from data prep file
dfIncidents = package.run('Incidents.dprep', dataflow_idx=0)
# dfIncidents = pd.read_csv('allIncidents.csv', encoding="ISO-8859-1")
dfRequests = package.run('Requests.dprep', dataflow_idx=0)
# dfIncidents = package.run('IncidentsCleaned.dprep', dataflow_idx=0)
# dfRequests = package.run('RequestsCleaned.dprep', dataflow_idx=0)

# Load dataset from file
# dfIncidents = pd.read_csv('./data/endava_tickets/all_incidents.csv')
# dfRequests = pd.read_csv('./data/endava_tickets/all_requests.csv')
#####################


In [3]:
print('dfIncidents shape:',dfIncidents.shape)
print('dfRequests shape:',dfRequests.shape)
dfIncidents

dfIncidents shape: (14186, 9)
dfRequests shape: (38396, 9)


,category,sub_category1,sub_category2,title,body,impact,urgency,business_service,ticket_type
0,Endava Application Support,Functionality,Access,Acces cladirea E5 - No access - IASI,From:Alina.Morosac@endava.com\r\nTo:ITS@endava...,3 - Low,1 - High,Endava Physical Access Service,Incident
1,Endava Application Support,Functionality,Error,Connection issues on the Worlpday network with...,From:Andreea.Toderean@endava.com\r\nTo:ITS@end...,3 - Low,2 - Medium,Endava LAN and WIFI access Service,Incident
2,Endava Application Support,Functionality,Access,Cannot access https://cds/,From:Mihai.Dinu@endava.com\r\nTo:ITS@endava.co...,3 - Low,3 - Low,Endava CDS Service,Incident
3,Endava Application Support,User Account,Authentication,Email address shown as not valid - Name change...,From:Myrielle.Lorscheid@endava.com\r\nTo:ITS@e...,3 - Low,2 - Medium,Endava Active Directory Service,Incident
4,Infrastructure Services,Managed Servers,Storage Device,DPM: Critical alert: UKIXBKP01,"From: DPM Service \r\r\nSent: Tuesday, January...",3 - Low,3 - Low,Endava Backup Service,Incident
5,Endava Application Support,Data,Content,Timesheet code spelling mistake in Oracle,From:Bradley.Howard@endava.com\r\nTo:ITS@endav...,3 - Low,3 - Low,Endava Oracle Fusion HCM,Incident
6,Endava Application Support,Reporting,Access,Timesheet Oracle and Annual Leave,From:Roman.Pavlic@endava.com\r\nTo:ITS@endava....,3 - Low,3 - Low,Endava Oracle Fusion HCM,Incident
7,Endava Application Support,Functionality,Error,Endava Jira Service unavailable,From:Roman.Bulat@endava.com\r\nTo:ITS@endava.c...,2 - Medium,1 - High,Endava Jira Service,Incident
8,Endava Application Support,Reporting,Error,P3O report in Oracle BI not working,From:Marcel.Morozan@endava.com\r\nTo:ITS@endav...,3 - Low,3 - Low,Endava Oracle Fusion Finance (FPP) Service,Incident
9,Endava Application Support,User Account,Authentication,No more access to Outlook,From:hans-peter.forster@arcor.de\r\r\nTo:its@e...,3 - Low,2 - Medium,Endava Email Service,Incident


In [4]:
dfRequests

,title,business_service,category,sub_category1,sub_category2,ticket_type,body,impact,urgency
0,Endava Uptime Reporting ...,Endava Reporting Service,Service Management,reporting,Capacity Report,Request,\r\n\r\nFrom: Capacity Reporting \r\nSent: Mon...,3 - Low,3 - Low
1,Add to DL,Endava Email Service,Endava Application Support,Access Management,,Request,"From: Edward Butler \r\nSent: Tuesday, October...",3 - Low,3 - Low
2,PO 604000002572 - Mihai Marinescu- AFi,Endava Desktop Support Service,Endava Desktop Services,Desktop Hardware,Add/Install,Request,"\r\nFrom: UKM Purchasing \r\nSent: Tuesday, De...",3 - Low,3 - Low
3,Set up and install Latest Apple OSX and prep...,Endava Non Standard IT Service,Endava Desktop Services,Desktop OS/Application,Add/Install,Request,"Hi ITS,\r\n\r\nPlease open tickets and assign ...",3 - Low,3 - Low
4,employment changes CDS,Endava CDS Service,Endava Application Support,Account Management,Add/Install,Request,From: Miroslav Stipanovic \r\nSent: 06 Decembe...,3 - Low,3 - Low
5,set 703 - SPAM,Endava Spam Filter Service,Endava Application Support,Account Management,Configuration Change,Request,"From: Savannah Gallo \r\nSent: Monday, July 18...",3 - Low,3 - Low
6,"""Access to PAS|Ivan Danev""",Endava Oracle Fusion Finance (FPP) Service,Endava Application Support,Application Management,Add/Install,Request,From:AMOraclePASSupport@endava.com\r\nTo:ITS@e...,3 - Low,3 - Low
7,"""Monthly PAS Activities Calendar"" permissions",Endava Email Service,Endava Application Support,Account Management,Configuration Change,Request,From:Jennifer.Oloyede@endava.com\r\nTo:Rob.Mac...,3 - Low,3 - Low
8,"""Security"" DL membership",Endava Email Service,Endava Application Support,Application Management,Add/Install,Request,"\r\n\r\nFrom: David Feltham \r\nSent: Tuesday,...",3 - Low,3 - Low
9,#Leaver form for Adrian Zburatura - leave dat...,Endava MSD Tool Service,Endava Application Support,Account Management,Leaver,Request,"From: HR NetAdmin \r\nSent: Friday, July 7, 20...",3 - Low,3 - Low


In [5]:
# Reorder columns
columnsOrder = [
    'title', 'body', 'ticket_type', 'category',
    'sub_category1', 'sub_category2', 'business_service',
    'urgency', 'impact'
]
dfIncidents = dfIncidents[columnsOrder]
dfRequests = dfRequests[columnsOrder]

In [6]:
dfIncidents

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,Acces cladirea E5 - No access - IASI,From:Alina.Morosac@endava.com\r\nTo:ITS@endava...,Incident,Endava Application Support,Functionality,Access,Endava Physical Access Service,1 - High,3 - Low
1,Connection issues on the Worlpday network with...,From:Andreea.Toderean@endava.com\r\nTo:ITS@end...,Incident,Endava Application Support,Functionality,Error,Endava LAN and WIFI access Service,2 - Medium,3 - Low
2,Cannot access https://cds/,From:Mihai.Dinu@endava.com\r\nTo:ITS@endava.co...,Incident,Endava Application Support,Functionality,Access,Endava CDS Service,3 - Low,3 - Low
3,Email address shown as not valid - Name change...,From:Myrielle.Lorscheid@endava.com\r\nTo:ITS@e...,Incident,Endava Application Support,User Account,Authentication,Endava Active Directory Service,2 - Medium,3 - Low
4,DPM: Critical alert: UKIXBKP01,"From: DPM Service \r\r\nSent: Tuesday, January...",Incident,Infrastructure Services,Managed Servers,Storage Device,Endava Backup Service,3 - Low,3 - Low
5,Timesheet code spelling mistake in Oracle,From:Bradley.Howard@endava.com\r\nTo:ITS@endav...,Incident,Endava Application Support,Data,Content,Endava Oracle Fusion HCM,3 - Low,3 - Low
6,Timesheet Oracle and Annual Leave,From:Roman.Pavlic@endava.com\r\nTo:ITS@endava....,Incident,Endava Application Support,Reporting,Access,Endava Oracle Fusion HCM,3 - Low,3 - Low
7,Endava Jira Service unavailable,From:Roman.Bulat@endava.com\r\nTo:ITS@endava.c...,Incident,Endava Application Support,Functionality,Error,Endava Jira Service,1 - High,2 - Medium
8,P3O report in Oracle BI not working,From:Marcel.Morozan@endava.com\r\nTo:ITS@endav...,Incident,Endava Application Support,Reporting,Error,Endava Oracle Fusion Finance (FPP) Service,3 - Low,3 - Low
9,No more access to Outlook,From:hans-peter.forster@arcor.de\r\r\nTo:its@e...,Incident,Endava Application Support,User Account,Authentication,Endava Email Service,2 - Medium,3 - Low


In [7]:
# Merge incidents and requests datasets
dfTickets = dfRequests.append(
    dfIncidents,
    ignore_index=True)  # set True to avoid index duplicates
print(dfTickets.shape)

(52582, 9)


In [8]:
# Remove duplicates
columnsToDropDuplicates = ['body']
dfTickets = dfTickets.drop_duplicates(columnsToDropDuplicates)
print(dfTickets.shape)

(52570, 9)


In [9]:
# Merge 'title' and 'body' columns into single column 'body'
# dfTickets['body'] = (dfTickets['title']+
#   " " + dfTickets['body']).map(str)
# dfTickets = dfTickets.drop(['title'], axis=1)

## Remove text with regex

In [10]:
# Select columns for cleaning
columnsToClean = ['body', 'title']

# Create list of regex to remove sensitive data
# Clean dataset and remove sensitive data
cleanDataset(dfTickets, columnsToClean, getRegexList())


,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,endava uptime reporting ...,capacity reporting for september ...,Request,Service Management,reporting,Capacity Report,Endava Reporting Service,3 - Low,3 - Low
1,add to dl,fw continuous delivery meetup hi i...,Request,Endava Application Support,Access Management,,Endava Email Service,3 - Low,3 - Low
2,po mihai marinescu afi,po hello x kings...,Request,Endava Desktop Services,Desktop Hardware,Add/Install,Endava Desktop Support Service,3 - Low,3 - Low
3,set up and install latest apple osx and prep...,hi its please open tickets and assign to m...,Request,Endava Desktop Services,Desktop OS/Application,Add/Install,Endava Non Standard IT Service,3 - Low,3 - Low
4,employment changes cds,re employment changes hi iulia i m...,Request,Endava Application Support,Account Management,Add/Install,Endava CDS Service,3 - Low,3 - Low
5,set spam,fw set hi i received this ema...,Request,Endava Application Support,Account Management,Configuration Change,Endava Spam Filter Service,3 - Low,3 - Low
6,access to pas ivan danev,fw request ritm access to pas ...,Request,Endava Application Support,Application Management,Add/Install,Endava Oracle Fusion Finance (FPP) Service,3 - Low,3 - Low
7,monthly pas activities calendar permissions,re query hi its any endavan shou...,Request,Endava Application Support,Account Management,Configuration Change,Endava Email Service,3 - Low,3 - Low
8,security dl membership,fw security dl membership hi i...,Request,Endava Application Support,Application Management,Add/Install,Endava Email Service,3 - Low,3 - Low
9,leaver form for adrian zburatura leave dat...,leaver form for adrian zburatura leave...,Request,Endava Application Support,Account Management,Leaver,Endava MSD Tool Service,3 - Low,3 - Low



## Remove all non english words + names + blacklisted


In [11]:
# Firstly load english words dataset and names dataset
# dfWordsEn = package.run('EnglishWords.dprep', dataflow_idx=0)
# dfWordsEn = package.run('EnglishWordsAlpha.dprep', dataflow_idx=0)
# dfWordsEn = package.run('EnglishWordsMerged.dprep', dataflow_idx=0)
dfWordsEn = package.run('WordsEn.dprep', dataflow_idx=0)
dfFirstNames = package.run('FirstNames.dprep', dataflow_idx=0)
dfBlackListWords = package.run('WordsBlacklist.dprep', dataflow_idx=0)

In [12]:
# Transform all words to lower case
dfWordsEn['Line'] = dfWordsEn['Line'].str.lower()
dfFirstNames['Line'] = dfFirstNames['Line'].str.lower()
dfBlackListWords['Line'] = dfBlackListWords['Line'].str.lower()

In [13]:
# Merge datasets removing names from English words dataset
print("Shape before removing first names from\
    english words dataset: "+str(dfWordsEn.shape))
dfWords = dfWordsEn.merge(
    dfFirstNames.drop_duplicates(),
    on=['Line'], how='left', indicator=True)

# Select words without names only
dfWords = dfWords.loc[dfWords['_merge'] == 'left_only']
print("Shape after removing first names from \
english words dataset: "+str(dfWords.shape))
dfWords = dfWords.drop("_merge", axis=1)  # Drop merge indicator column

Shape before removing first names from    english words dataset: (109583, 1)
Shape after removing first names from english words dataset: (108740, 2)


In [14]:
# Merge datasets removing blacklisted words
print("Shape before removing blacklisted\
    words from english ords dataset: "+str(dfWords.shape))
dfWords = dfWords.merge(
    dfBlackListWords.drop_duplicates(),
    on=['Line'], how='left', indicator=True)

# Select words
dfWords = dfWords.loc[dfWords['_merge'] == 'left_only']
print("Shape after removing blacklisted \
words from english words dataset: "+str(dfWords.shape))

Shape before removing blacklisted    words from english ords dataset: (108740, 1)
Shape after removing blacklisted words from english words dataset: (108333, 2)


In [15]:
print(dfTickets.shape)
# Remove non english words and names
dfTickets['body'] = dfTickets['body'].apply(
    lambda emailBody: removeNonEnglish(emailBody, dfWords['Line']))
print(dfTickets.shape)
dfTickets['title'] = dfTickets['title'].apply(
    lambda emailBody: removeNonEnglish(emailBody, dfWords['Line']))
print(dfTickets.shape)

(52570, 9)
(52570, 9)
(52570, 9)


In [16]:
# Remove empty strings and null rows after removing non english words
print("Before removing empty: " + str(dfTickets.shape))
dfTickets = dfTickets[dfTickets.body != " "]
dfTickets = dfTickets[dfTickets.body != ""]
dfTickets = dfTickets[~dfTickets.body.isnull()]
print("After removing empty: " + str(dfTickets.shape))

# Remove duplicates x2
columnsToDropDuplicates = ['body']
dfTickets = dfTickets.drop_duplicates(columnsToDropDuplicates)
print("After removing duplicates:",dfTickets.shape)

Before removing empty: (52570, 9)
After removing empty: (52531, 9)
After removing duplicates: (48377, 9)


## Data encryption and anonymization using LabelEncoder 

In [17]:
# Select columns for encryption
columnsToEncrypt = [
    'category', 'sub_category1', 'sub_category2',
    'business_service', 'urgency',
    'impact', 'ticket_type'
]

# Encrypt data for each of selected columns
dfTickets = encryptColumnsCollection(dfTickets, columnsToEncrypt)

## Save dataset and removed words to csv

In [ ]:
# Save cleaned and encrypted dataset back to csv without indexes
dfTickets.to_csv('all_tickets.csv', index=False, index_label=False)

In [ ]:
sortedRemovedWordsList = np.sort(removedWordsList)
dfx = pd.DataFrame(sortedRemovedWordsList)
dfx.to_csv("removed_words.csv", index=False, index_label=False)